<a href="https://colab.research.google.com/github/PashaIanko/Kaggle.Restaurant-Revenue-Prediction/blob/main/6.neural_networks_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
from google.colab import drive
import pandas as pd
import numpy as np
import os

In [2]:
# PEP 8
%%capture
!pip install pycodestyle
!pip install --index-url https://test.pypi.org/simple/ nbpep8

from nbpep8.nbpep8 import pep8

# Import module files

In [3]:
%%capture
git_download_path = \
'https://raw.githubusercontent.com/PashaIanko/Kaggle.Restaurant-Revenue-Prediction/main/'

files = [
    'sklearn_utils.py',
    'path_manager.py'
]

def download_files(dict_):
    for file, url in dict_.items():
        !wget {file} {url} {file}


download_files(
    {
        file: git_download_path + file for file in files
    }
)

pep8(_ih)

In [4]:
import importlib
import sklearn_utils
import path_manager


def reload_modules(modules):
    for m in modules:
        importlib.reload(m)


reload_modules(
    [
        sklearn_utils
    ]
)

from sklearn_utils import print_model_cv_scores
from path_manager import PathManager

pep8(_ih)

cell_content.py:17:1: E402 module level import not at top of file
cell_content.py:18:1: E402 module level import not at top of file



# Setup paths

In [5]:
drive.mount('/content/gdrive')
gdrive_path = '/content/gdrive/MyDrive/'

COMPETITION_PATH = gdrive_path + 'ML/Competitions/5.RestaurantRevenue/'
PREPROC_TRIAL = 2
MODELS_TRIAL = 2

pep8(_ih)

Mounted at /content/gdrive



In [6]:
manager = PathManager(
    competition_path=COMPETITION_PATH,
    preprocessing_trial=PREPROC_TRIAL,
    models_trial=MODELS_TRIAL
)

pep8(_ih)

# Download models

In [7]:
models = manager.load_models(
    [
        'FFNN'
    ]
)

In [8]:
models

{'FFNN': <keras.engine.sequential.Sequential at 0x7f3f6f5a76d0>}

# Download data

In [9]:
df_test = pd.read_csv(
    os.path.join(
        manager.data_trial_path,
        'test_processed.csv'
    ),
    index_col=[0]
)

# subsample from train.csv for testing
df_test_sample = pd.read_csv(
    os.path.join(
        manager.data_trial_path,
        'test_sample_processed.csv'
    ),
    index_col=[0]
)

df_trainval = pd.read_csv(
    os.path.join(
        manager.data_trial_path,
        'trainval_sample_processed.csv'
    ),
    index_col=[0]
)

X_train_val = df_trainval.values[:, :-1]
Y_train_val = df_trainval.values[:, -1]

X_test_sample = df_test_sample.values[:, :-1]
Y_test_sample = df_test_sample.values[:, -1]

# Data treatment (follow the same procedure as in neural_networks_preparation.ipynb)

In [10]:
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()
scaler.fit(X_train_val)
X_test_sample_scaled = scaler.transform(X_test_sample)

pep8(_ih)

In [11]:
np.max(X_test_sample_scaled)

1.6666666666666665

# Report test sample performance

In [12]:
final_model = models['FFNN']

In [13]:
from sklearn.metrics import mean_squared_error

test_sample_predictions = final_model.predict(
    X_test_sample_scaled
)

In [14]:
mean_squared_error(
    test_sample_predictions,
    Y_test_sample
)

0.38359666013648086

# Fitting on test sample before submission

In [16]:
final_model.fit(
    X_test_sample_scaled,
    Y_test_sample,
    epochs=2
)

Epoch 1/2
1/1 [==============================] - 0s 27ms/step - loss: 0.3440
Epoch 2/2
1/1 [==============================] - 0s 16ms/step - loss: 0.3317


# Submitting results

In [17]:
df_test.head(1)

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,43
0,0.995545,-1.032819,-0.285964,-0.26814,-0.383325,-0.807152,-0.633614,-0.182156,-0.619713,-0.217343,...,-0.497002,-0.606797,-0.556056,-0.652386,0.0,0.0,1.0,0.0,0.020451,0.693147


In [18]:
submission_data = df_test.values[:, :-1]
submission_data.shape

(100000, 43)

In [19]:
submission_data_scaled = scaler.transform(
    submission_data
)

In [21]:
submission_data_scaled[:, 0]

array([0.83333333, 0.83333333, 0.94444444, ..., 0.88888889, 0.94444444,
       0.77777778])

In [23]:
final_predictions = np.exp(
    final_model.predict(submission_data_scaled)
)
final_predictions[:5]

array([[3953342.8],
       [4843340. ],
       [4010883.5],
       [9914572. ],
       [6559134. ]], dtype=float32)

In [26]:
df_submission = pd.DataFrame(
    final_predictions.ravel(),
    columns=['Prediction']
)
df_submission.index.rename('Id', inplace=True)
df_submission.head()

,Prediction
Id,
0,3953342.75
1,4843340.00
2,4010883.50
3,9914572.00
4,6559134.00


In [27]:
final_model_name = 'FFNN_retrained'

submission_path = os.path.join(
    manager.models_trial_path,
    final_model_name,
    'submission.csv'
)

manager.save_models(
    {
        final_model_name: final_model
    }
)
df_submission.to_csv(submission_path)